In [ ]:
# Load need packages and provide Metrics App names

!pip install semantic-link --q
 
import pandas as pd
import sempy.fabric as fabric
from datetime import datetime,date,timedelta

#Update with names from your install of the Fabric Metrics Apps
#Note - This code works with V1.5 of Metrics App (March 2024). Breaking changes possible in later updates.
MetricsWS = "FabricMetricsApp_March2024"
MetricsModel = "Fabric Capacity Metrics"

In [ ]:
# Get Capacities, write to daily csv, overwrite Capacities Table
mssparkutils.fs.mkdirs("Files/Capacities/")
df_capacities = fabric.evaluate_dax(workspace=MetricsWS, dataset=MetricsModel, dax_string="EVALUATE Capacities")
df_capacities.columns = df_capacities.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
df_capacities.columns = df_capacities.columns.str.replace(' ', '_')
#Need CU for each SKU to enable utilization calculation
skus = {'SKU': ['P1', 'P2', 'P3', 'P4', 'P5', 'F2', 'F4', 'F8', 'F16', 'F32', 'F64', 'F128', 'F256', 'F512', 'F1024', 'F2048', 'FT1', 'A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'DCT1', 'EM1', 'EM2', 'EM3'], 'CUperSecond': [64, 128, 256, 512, 1024, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 64, 8, 16, 32, 64, 128, 256, 512, 1024, 64, 8, 16, 32], 'CUperHour': [230400, 460800, 921600, 1843200, 3686400, 7200, 14400, 28800, 57600, 115200, 230400, 460800, 921600, 1843200, 3686400, 7372800, 230400, 28800, 57600, 115200, 230400, 460800, 921600, 1843200, 3686400, 230400, 28800, 57600, 115200]}
df_skus = pd.DataFrame(skus)
df_capacities = df_capacities.merge(df_skus, left_on="sku", right_on="SKU")
df_capacities = df_capacities.drop('SKU', axis=1)
filename = 'Capacities_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
df_capacities.to_csv("/lakehouse/default/Files/Capacities/" + filename)
spk_capacities = spark.createDataFrame(df_capacities)
spk_capacities.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable('Capacities')

In [ ]:
# Get Items, write to daily csv, overwrite Items Table
mssparkutils.fs.mkdirs("Files/Items/")
df_items = fabric.evaluate_dax(workspace=MetricsWS, dataset=MetricsModel, dax_string="EVALUATE Items")
df_items.columns = df_items.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
df_items.columns = df_items.columns.str.replace(' ', '_')
df_items.drop_duplicates(subset=['ItemId'], keep='first', inplace=True)
filename = 'Items_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
df_items.to_csv("/lakehouse/default/Files/Items/" + filename)
spk_items = spark.createDataFrame(df_items)
spk_items.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable('Items')

In [ ]:
# UsageData
# Get list of capacities
lst_capacities = df_capacities['capacityId'].tolist()

# Create dir if doesn't exist
mssparkutils.fs.mkdirs("Files/UsageData/")

# Get max date from current delta table (to avoid loading duplicate days)
try:
    df_max = spark.sql(f'''
    SELECT MAX(Date) as MaxDate
    FROM usage;
    '''
    )
    maxdate = df_max.first()['MaxDate']
except:
    maxdate = datetime.today() + timedelta(days=-6)
    #Note - pulling too much data may result in 64 Mb error; reduce number of days for initial load
maxdateforDAX = maxdate.strftime('%Y,%m,%d')

if maxdate.date() < (datetime.today() + timedelta(days=-1)).date():

    # Get data for each capacity, write daily csv and append delta
    for capacity in lst_capacities:   
        querytext = '''\
                    DEFINE MPARAMETER 'CapacityID' =  "{capID}"
                    var Yesterday = TODAY() - 1

                    EVALUATE
                    CALCULATETABLE(MetricsByItemandOperationandHour, MetricsByItemandOperationandHour[Date] <= Yesterday && MetricsByItemandOperationandHour[Date] > DATE({MD}))
                    '''.format(capID=capacity, MD=maxdateforDAX)
        df_hourly = fabric.evaluate_dax(workspace=MetricsWS, dataset=MetricsModel, dax_string=querytext)
        if len(df_hourly) >= 1:
            df_hourly.columns = df_hourly.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
            df_hourly.columns = df_hourly.columns.str.replace(' ', '_')
            df_hourly.rename(columns={'Throttling_(min)': 'ThrottlingMin'}, inplace=True)
            filename = capacity + '_Usage_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
            df_hourly.to_csv("/lakehouse/default/Files/UsageData/" + filename)
            spk_usage = spark.createDataFrame(df_hourly)
            spk_usage.write.mode("append").format("delta").option("overwriteSchema", "true").saveAsTable('Usage')
else:
    print("Data already loaded")

In [ ]:
# ThrottlingData
# Get list of capacities
lst_capacities = df_capacities['capacityId'].tolist()

# Create dir if doesn't exist
mssparkutils.fs.mkdirs("Files/ThrottlingData/")

# Get max date from current delta table (to avoid loading duplicate days)
try:
    df_max = spark.sql(f'''
    SELECT MAX(Date) as MaxDate
    FROM throttling;
    '''
    )
    maxdate = df_max.first()['MaxDate']
except:
    maxdate = datetime.today() + timedelta(days=-6)
maxdateforDAX = maxdate.strftime('%Y,%m,%d')

if maxdate.date() < (datetime.today() + timedelta(days=-1)).date():

    # Get data for each capacity, write daily csv and append delta
    for capacity in lst_capacities:   
        querytext = '''\
                    DEFINE
                    MPARAMETER 'CapacityID' = "{capID}"
                    VAR yesterday =
                        FILTER(ALL('Dates'[Date] ), 'Dates'[Date] < TODAY() && 'Dates'[Date] > DATE({MD}) )

                    EVALUATE
                    SUMMARIZECOLUMNS(
                        'Dates'[Date],
                        'TimePoints'[Start of Hour],
                        yesterday,
                        "IntDelay", ROUND( 'All Measures'[Dynamic InteractiveDelay %] * 100, 2 ),
                        "IntReject", ROUND( 'All Measures'[Dynamic InteractiveRejection %] * 100, 2 ),
                        "BackReject", ROUND( 'All Measures'[Dynamic BackgroundRejection %] * 100, 2 )
                    )
                    '''.format(capID=capacity, MD=maxdateforDAX)
        df_throttling = fabric.evaluate_dax(workspace=MetricsWS, dataset=MetricsModel, dax_string=querytext)
        if len(df_throttling) >= 1:
            df_throttling.columns = df_throttling.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
            df_throttling.columns = df_throttling.columns.str.replace(' ', '_')
            df_throttling['capacityId'] = capacity
            filename = capacity + '_throttling_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
            df_throttling.to_csv("/lakehouse/default/Files/ThrottlingData/" + filename)
            spk_throttle = spark.createDataFrame(df_throttling)
            spk_throttle.write.mode("append").format("delta").option("overwriteSchema", "true").saveAsTable('Throttling')

else:
    print("Data already loaded")

In [ ]:
# DateHour Table
import pandas as pd
from datetime import datetime
import numpy as np
import math
from pandas.tseries.offsets import MonthEnd, QuarterEnd

# Create a list of dates from 2020 to 2024
start_date = datetime(2024, 2, 24)
end_date = pd.to_datetime("today") #datetime(2024, 12, 31)
dates = pd.date_range(start=start_date, end=end_date, freq = '1H')

# Create a dataframe with the desired columns
df = pd.DataFrame({
    'DateHour': dates,
    'Hour': dates.strftime('%H').astype('int'),
    'Year': dates.year,
    'MonthNumber': dates.month,
    'Month': dates.strftime('%B'),
    'MonthShort': dates.strftime('%b'),
    'Day': dates.strftime('%A').astype('str'),
    'DayShort': dates.strftime('%a'),
    'DayOfWeek': dates.weekday,
    'DayOfYear': dates.strftime('%j').astype('int'),
    'WeekOfYear': dates.strftime('%W').astype('int'),
    'YearQuarter': dates.year.astype('str') + "Q" + dates.quarter.astype('str'),
    'YearWeek': dates.strftime('%Y%W'),
    'EOM': pd.to_datetime(dates, format="%Y%m") + MonthEnd(0),
    'EOQ': pd.to_datetime(dates, format="%Y%m") + QuarterEnd(0)
})

# Add date column
df['Date'] = pd.to_datetime(pd.to_datetime(df['DateHour']).dt.date)

# Create index columns from today
today = datetime.today()
df['DaysFromToday'] = (df['Date']-today).dt.days
df['WeeksFromToday'] = ((df['Date']-today).dt.days/7).astype('int')
df['MonthsFromToday'] = ((df['Year']-today.year) * 12 + ( df['MonthNumber']-today.month))
df['QtrsFromToday'] = (df['MonthsFromToday'].astype('int')/3).astype('int')
df['YearsFromToday'] = df['Year']-today.year

# Add Is Working Day

df['WorkingDay'] = np.where((df['DayOfWeek'] != 5) & (df['DayOfWeek'] != 6), True, False)

# Define function to add zero-width spaces for month and day names
def AddZWS(row, col, num):
    return (num-row[col]) * chr(8203)

# Add ZWS columns for month and day
df['MonthZWS'] = df.apply(AddZWS, num=12, col='MonthNumber', axis=1) + df['Month']
df['DayZWS'] = df.apply(AddZWS, num=6, col='DayOfWeek', axis=1) + df['Day']

# Convert pandas DF to spark DF and write to delta
# Remove spaces from column names (if any)
df.columns = df.columns.str.replace(' ', '')

# Create Spark Datafram and Load to Delta Table
sparkDF = spark.createDataFrame(df)
sparkDF.write.mode("overwrite").format("delta").option("overwriteSchema", "true").saveAsTable("DateHour")